In [1]:
import json
import threading
import queue
import time
from datetime import datetime
import random
import string
from typing import List

def generate_random_string(length: int = 30) -> str:
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

def create_user(index: int) -> dict:
    return {
        "username": f"user{index}",
        "uid": generate_random_string(),
        "dateOfJoining": datetime.utcnow().isoformat()[:-3] + 'Z',
        "RoomId": [],
        "password": generate_random_string()
    }

def worker(work_queue: queue.Queue, results: List[dict], lock: threading.Lock):
    while True:
        try:
            index = work_queue.get_nowait()
            user = create_user(index)
            with lock:
                results.append(user)
            work_queue.task_done()
        except queue.Empty:
            break

def write_users_to_file(num_users: int = 1000, num_threads: int = 10):
    work_queue = queue.Queue()
    results = []
    lock = threading.Lock()
    
    for i in range(num_users):
        work_queue.put(i)
    
    threads = []
    for _ in range(num_threads):
        thread = threading.Thread(target=worker, args=(work_queue, results, lock))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()
        
    with open('data.json', 'w') as f:
        json.dump(results, f, indent=2)


In [2]:
def calculate_time_taken():
    start = time.time()
    write_users_to_file(num_users=1000000)
    end = time.time()
    print(f"Time taken: {end - start} seconds")
    print(f"Time taken: {(end - start)/60} minutes")
    print(f"Time taken: {(end - start)/3600} hours")    

In [3]:
calculate_time_taken()

Time taken: 140.3171784877777 seconds
Time taken: 2.338619641462962 minutes
Time taken: 0.0389769940243827 hours


In [3]:
# script to validte the json data array generated format and weather the whole data is written to the file or not
with open('data.json', 'r') as f:
    data = json.load(f)
    print(len(data))


10000000


In [4]:
# is the json data array generated is in the correct format
print(data[9999999])


{'username': 'user9999999', 'uid': 'hiSdrFg1sc6VsjhHZ6wIv5kYPYFiNu', 'dateOfJoining': '2024-12-26T16:39:57.833Z', 'RoomId': [], 'password': 'Uc4SNvpATIijzCt1910RkZMqIi6TMY'}
